# Find Duplicate Documents
in this notebook we will use the Text_Deduplicater utility class to:
* find problematic texts in a single corpus
* compare texts for different corpora 
* build a large corpus by merging several distinct corpora and excluding duplicate documents. 

In [1]:
%load_ext autoreload
%autoreload 2

In [39]:
import logging 
import random
import os
import re
import pickle
from tqdm import tqdm_notebook
from cltk.corpus.readers import get_corpus_reader

In [3]:
import sys
import inspect
from pathlib import Path 
currentdir = Path.cwd()
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
from mlyoucanuse.text_deduplicater import TextDeduplicater

In [4]:
logging.basicConfig(level=logging.INFO)

In [5]:
# You may need to install the tesserae corpora
# from cltk.corpus.utils.importer import CorpusImporter
# corpus_importer = CorpusImporter('latin')
# #corpus_importer.list_corpora
# corpus_importer.import_corpus('latin_text_tesserae')

## Load our Corpus readers

In [6]:
perseus_latin_reader = get_corpus_reader(corpus_name='latin_text_perseus', language='latin')
latin_library_reader = get_corpus_reader(corpus_name='latin_text_latin_library', language='latin')
tesserae_reader = get_corpus_reader(corpus_name='latin_text_tesserae', language='latin')

## Create our Text Deduplicator

In [7]:
deduper = TextDeduplicater()

## Simple proof of the Dedupe functionality
Add Caesar docs and pick one of the files at random and add it as a duplicate

In [8]:
caesar = [file for  file in latin_library_reader._fileids if 'caesar' in file]
print(caesar)
deduper = TextDeduplicater()
for file in tqdm_notebook(caesar):
    text = list(latin_library_reader.docs( file ))[0]
    deduper.add_document(file, text)
dupe_file = random.choice(caesar)
text = list(latin_library_reader.docs(dupe_file ))[0]
deduper.add_document(dupe_file, text)
print(f'Unique doc names: {deduper.get_unique_doc_names()}')
print(f'Duplicate doc names: {deduper.get_possible_duplicate_doc_names()}')

['caesar/alex.txt', 'caesar/bc1.txt', 'caesar/bc2.txt', 'caesar/bc3.txt', 'caesar/bellafr.txt', 'caesar/gall1.txt', 'caesar/gall2.txt', 'caesar/gall3.txt', 'caesar/gall4.txt', 'caesar/gall5.txt', 'caesar/gall6.txt', 'caesar/gall7.txt', 'caesar/gall8.txt', 'caesar/hisp.txt', 'suetonius/suet.caesar.txt', 'xylander/caesar.txt']



Unique doc names: ['caesar/gall8.txt', 'caesar/hisp.txt', 'caesar/gall1.txt', 'caesar/bc3.txt', 'suetonius/suet.caesar.txt', 'xylander/caesar.txt', 'caesar/gall5.txt', 'caesar/bc2.txt', 'caesar/bc1.txt', 'caesar/alex.txt', 'caesar/bellafr.txt', 'caesar/gall7.txt', 'caesar/gall6.txt', 'caesar/gall3.txt', 'caesar/gall2.txt', 'caesar/gall4.txt']
Duplicate doc names: [('caesar/bc3.txt', 'caesar/bc3.txt')]


## Let's check the whole corpus contents
if a file has warnings about no trigrams found, it's an indicator that the file is empty.

In [9]:
# Reset the duplicator
deduper = TextDeduplicater()

for file in tqdm_notebook(latin_library_reader._fileids):
    text = list(latin_library_reader.docs(file))[0]
    deduper.add_document(file, text)

## Surprise, some problematic files just in one Corpus

In [10]:
latin_lib_problematic_files = deduper.get_possible_duplicate_doc_names()
latin_lib_problematic_files

[('lucan/lucan7.txt', 'lucan/lucan8.txt'),
 ('albertanus/albertanus.sermo4.txt', 'albertanus/albertanus.sermo3.txt'),
 ('albertanus/albertanus.sermo1.txt', 'albertanus/albertanus.sermo3.txt'),
 ('albertanus/albertanus.sermo2.txt', 'albertanus/albertanus.sermo1.txt'),
 ('albertanus/albertanus.sermo1.txt', 'albertanus/albertanus.sermo4.txt'),
 ('albertanus/albertanus.sermo2.txt', 'albertanus/albertanus.sermo4.txt'),
 ('albertanus/albertanus.sermo3.txt', 'albertanus/albertanus.sermo2.txt')]

## Let's check their similarity scores

In [11]:
print('Duplicate documents and similarity scores')
for doc_one, doc_two in latin_lib_problematic_files:
    print(doc_one, doc_two,  
    deduper.calculate_similarity( 
    list(latin_library_reader.docs(doc_one))[0],
    list(latin_library_reader.docs(doc_two))[0]))

Duplicate documents and similarity scores
lucan/lucan7.txt lucan/lucan8.txt 0.9987385682749921
albertanus/albertanus.sermo4.txt albertanus/albertanus.sermo3.txt 1.0
albertanus/albertanus.sermo1.txt albertanus/albertanus.sermo3.txt 1.0
albertanus/albertanus.sermo2.txt albertanus/albertanus.sermo1.txt 1.0
albertanus/albertanus.sermo1.txt albertanus/albertanus.sermo4.txt 1.0
albertanus/albertanus.sermo2.txt albertanus/albertanus.sermo4.txt 1.0
albertanus/albertanus.sermo3.txt albertanus/albertanus.sermo2.txt 1.0


## Note: the reported duplicate files are actually errors in the corpus
The Lucan file is nearly the same file twice, somehow.

In [20]:
for file_one, file_two in latin_lib_problematic_files:
    print(re.sub('\s+',' ', list(latin_library_reader.docs(file_one))[0][:200]))
    print(re.sub('\s+',' ', list(latin_library_reader.docs(file_two))[0][:200]))

Lucan Liber VII M. ANNAEI LVCANI BELLI CIVILIS LIBER SEPTIMVS Segnior, Oceano quam lex aeterna uocabat, luctificus Titan numquam magis aethera contra egit equos cursumque p
Lucan Liber VIII M. ANNAEI LVCANI BELLI CIVILIS LIBER OCTAVVS Segnior, Oceano quam lex aeterna uocabat, luctificus Titan numquam magis aethera contra egit equos cursumque
Albertano of Brescia [an error occurred while processing this directive] The Classics Page
Albertano of Brescia [an error occurred while processing this directive] The Classics Page
Albertano of Brescia [an error occurred while processing this directive] The Classics Page
Albertano of Brescia [an error occurred while processing this directive] The Classics Page
Albertano of Brescia [an error occurred while processing this directive] The Classics Page
Albertano of Brescia [an error occurred while processing this directive] The Classics Page
Albertano of Brescia [an error occurred while processing this directive] The Classics Page
Albertano of Bresci

## Let's merge the tuples so we can skip these later

In [33]:
files1, files2 = zip(* latin_lib_problematic_files)
latin_lib_problematic_files = files1 + files2
latin_lib_problematic_files

('lucan/lucan7.txt',
 'albertanus/albertanus.sermo4.txt',
 'albertanus/albertanus.sermo1.txt',
 'albertanus/albertanus.sermo2.txt',
 'albertanus/albertanus.sermo1.txt',
 'albertanus/albertanus.sermo2.txt',
 'albertanus/albertanus.sermo3.txt',
 'lucan/lucan8.txt',
 'albertanus/albertanus.sermo3.txt',
 'albertanus/albertanus.sermo3.txt',
 'albertanus/albertanus.sermo1.txt',
 'albertanus/albertanus.sermo4.txt',
 'albertanus/albertanus.sermo4.txt',
 'albertanus/albertanus.sermo2.txt')

## Let's take a closer look at the Latin library and Perseus corpora by looking for and comparing a shared file

In [21]:
latin_lib_plautus_file = [tmp for tmp in latin_library_reader._fileids if 'plautus' in tmp and 'menaechmi' in tmp]
latin_lib_plautus_file

['plautus/menaechmi.txt']

In [22]:
perseus_plautus_file = [tmp for tmp in perseus_latin_reader._fileids if 'plautus' in tmp and 'menaechmi' in tmp]
perseus_plautus_file

['plautus-titus-maccius__menaechmi__latin.json']

In [25]:
perseus_plautus_text = ''.join(list(perseus_latin_reader.paras(perseus_plautus_file)))
latin_lib_plautus_text = list(latin_library_reader.docs(latin_lib_plautus_file))[0]
tmp_deduper = TextDeduplicater()
tmp_deduper.add_document(perseus_plautus_file[0], perseus_plautus_text)
tmp_deduper.add_document(latin_lib_plautus_file[0], latin_lib_plautus_text)
tmp_deduper.get_possible_duplicate_doc_names(threshold=0.4)

[('plautus-titus-maccius__menaechmi__latin.json', 'plautus/menaechmi.txt')]

In [29]:
latin_lib_plautus_text[:400]

'Plautus: Menaechmi\n\t\t \n\t\t \n\t\t \n\t \n\t\n \n T. MACCI PLAVTI MENAECHMI \n\n \n\n PERSONAE \n \n\n PENICVLVS PARASITVS \nMENAECHMVS \nMENAECHMVS (SOSICLES) \nEROTIUM MERETRIX \nCYLINDRUS COCVS \nMESSENIO SERVVS \nANCILLA \nMATRONA \nSENEX \nMEDICVS\n \n\n ARGVMENTVM \n \n\n Mercator Siculus, quoi erant gemini filii, \nEi surrupto altero mors optigit. \nNomen surrepticii illi indit qui domist \nAvos paternus, facit Maenaechmum e '

In [30]:
perseus_plautus_text[:400]

'Salutem primum iam a principio propitiammihi atque vobis, spectatores, nuntio.atque adeo hoc argumentum graecissat, tamenita est adulescens; ipsus escae maxumaeQua de re aut cuius rei rerum omnium?Me neque isti male fecisse mulieri, quae me arguithanc domo ab se surrupuisse atque abstulisse deierat.sí ego intra aedis huius umquam, ubi habitat, penetravi pedem ,omnium hominum exopto ut fiam miseror'

## The Perseus corpus isn't satisfactory, so let's try merging the Latin library corpus and the Tesserae corpus

In [34]:
# Reset the duplicator
deduper = TextDeduplicater()

for file in tqdm_notebook(tesserae_reader._fileids):
    text = list(tesserae_reader.texts(file))[0]
    if len(text) > 0:
        deduper.add_document(file, text)    
for file in tqdm_notebook(latin_library_reader._fileids):
    if file not in latin_lib_problematic_files:
        text = list(latin_library_reader.docs(file))[0]
        if len(text) > 0:
            deduper.add_document(file, text)

## Find out which files are duplicate
Note: of course this does not exclude any files were several texts are put together.

In [37]:
dupes = deduper.get_possible_duplicate_doc_names()
single_files_to_exclude = [file_one for file_one, file_two in dupes]
single_files_to_exclude

['cicero/plancio.txt',
 'texts/silius_italicus.punica.part.14.tess',
 'texts/quintus_smyrnaeus.fall_of_troy.part.13.tess',
 'texts/jerome.vulgate.part.35.amos.tess',
 'lucretius/lucretius5.txt',
 'ilias.txt',
 'texts/tacitus.annales.part.16.tess',
 'texts/cicero.in_vatinium.tess',
 'texts/prudentius.psychomachia.tess',
 'caesar/gall6.txt',
 'texts/caesar.de_bello_gallico.part.4.tess',
 'texts/bede.historiam_ecclesiasticam_gentis_anglorum.part.3.tess',
 'ovid/ovid.fasti3.txt',
 'caesar/gall2.txt',
 'texts/cicero.de_finibus_bonorum_et_malorum.part.3.tess',
 'texts/statius.achilleid.tess',
 'texts/valerius_maximus.facta_et_dicta_memorabilia.part.2.tess',
 'tacitus/tac.hist3.txt',
 'texts/tacitus.historiae.part.2.tess',
 'lucretius/lucretius4.txt',
 'bible/revelation.txt',
 'bede/bede4.txt',
 'texts/silius_italicus.punica.part.17.tess',
 'cicero/verres.2.5.txt',
 'cicero/ver1.txt',
 'texts/tacitus.annales.part.12.tess',
 'texts/marcus_mincuius_felix.octavius.tess',
 'texts/tacitus.annales.

## Now we can use these distinct files to build a single larger corpus